In [198]:
import pandas as pd
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack


'''
create tf-ifd feature of for a given column of the patent_table_clean dataframe
'''

project_dir = os.path.abspath(os.path.join( '__file__' , '..', '..'))
project_dir
input_dir = os.path.join(project_dir, 'data', 'processed', 'sample')
df = pd.read_pickle(os.path.join(input_dir, 'patent_table_clean.pickle'))
dendogram_df = pd.DataFrame(index=df.index.copy())

cols = ['title', 'abstract'] 

df = df.fillna('')  # ignore nans
features_dict = {}
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words='english')
for col in cols:
    features_dict[col] = vectorizer.fit_transform(df[col])  # calc tf-idf

features_sparse_matrix = hstack(list(features_dict.values()))


In [199]:
LEVELS = [100,50,25,7]
dendogram_df = pd.concat([dendogram_df, pd.DataFrame(columns=LEVELS)], copy=False)


print("K-means...")
last_centroids = None
for i in range(len(LEVELS)):
    k = LEVELS[i]
    print("K-means for " + str(k) + " clusters")
    t0 = time.time()
    km = KMeans(n_clusters=k)
    if i == 0:
        km.fit(features_sparse_matrix)
        dendogram_df.loc[:, k] = km.labels_
    else:
        km.fit(last_centroids)
        for j in range(len(last_centroids)):
            dendogram_df.ix[dendogram_df[LEVELS[i-1]] == j, k] = km.labels_[j]
    last_centroids = km.cluster_centers_
    t_batch = time.time() - t0

# nkm = KMeans(n_clusters=7)
# nkm.fit(last_centroids)
    
print("done.")

K-means...
K-means for 100 clusters
K-means for 50 clusters
K-means for 25 clusters
K-means for 7 clusters
done.


In [206]:
g = dendogram_df.loc[:,7].unique()
g

array([2, 0, 6, 4, 5, 1, 3], dtype=object)